In [1]:
import requests
import os
import sys
import pandas as pd
import numpy as np

# import key sklearn ml algos
# import key sklearn metrics
# import 

from typing import Optional
import glob
from scipy import stats
# Add the parent directory of this notebook to sys.path
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
parent_dir = os.path.dirname(notebook_dir)
sys.path.append(parent_dir)

from project_tools import project_utils, project_class

import datetime
import json
from tqdm.notebook import tqdm
import gc
# import ds_utils
import random
import matplotlib.pyplot as plt
%matplotlib inline

from importlib import reload
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

# Or for more precise control
pd.set_option('display.float_format', '{:.0f}'.format)


# load data

In [3]:
!ls ../data/big-data-derby-2022/

nyra_2019_complete.csv	nyra_start_table.csv
nyra_race_table.csv	nyra_tracking_table.csv


In [4]:
start_df = pd.read_csv('../data/big-data-derby-2022/nyra_start_table.csv')
# race_df = pd.read_csv('../data/big-data-derby-2022/nyra_race_table.csv')
# track_df = pd.read_csv('../data/big-data-derby-2022/nyra_tracking_table.csv')
# complete_df = pd.read_csv('../data/big-data-derby-2022/nyra_2019_complete.csv')


/tmp/ipykernel_91281/4252970222.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  complete_df = pd.read_csv('../data/big-data-derby-2022/nyra_2019_complete.csv')


In [13]:
start_cols = ['track_id', 'race_date', 'race_number', 'program_number', 'weight_carried',
             'jockey', 'odds', 'position_at_finish']

start_df.columns = start_cols

In [5]:
start_df.shape

(14914, 8)

In [6]:
race_df.shape

(2000, 10)

In [7]:
track_df.shape

(5228430, 7)

In [15]:
start_df.head(3)

,track_id,race_date,race_number,program_number,weight_carried,jockey,odds,position_at_finish
0,AQU,2019-01-01,1,2,120,Junior Alvarado,295,3
1,AQU,2019-01-01,1,3,118,Jose Lezcano,180,4
2,AQU,2019-01-01,1,4,123,Jomar Garcia,1280,5


In [12]:
race_df.head(3)

,track_id,race_date,race_number,distance_id,course_type,track_condition,run_up_distance,race_type,purse,post_time
0,AQU,2019-01-01,1,650,D,MY,36,AOC,80000,1220
1,AQU,2019-01-01,2,600,D,MY,48,MCL,41000,1250
2,AQU,2019-01-01,3,550,D,MY,54,MCL,35000,121


In [20]:
track_df.head(3)

,track_id,race_date,race_number,program_number,trakus_index,latitude,longitude
0,AQU,2019-01-01,9,6,72,41,-74
1,AQU,2019-01-01,9,6,73,41,-74
2,AQU,2019-01-01,9,6,74,41,-74


In [ ]:
race_df

In [38]:
rn = 10
rd = "2019-03-22"
tid = "AQU"
pn = "3  "
cond = (track_df['race_number']==1) & (track_df['race_date']==rd) & \
        (track_df['track_id']==tid) # & (track_df['program_number']==pn)
cdf = track_df[cond]#.sample(100).sort_values(by='trakus_index')

In [39]:
cdf['program_number'].value_counts()

program_number
1      324
4      324
5      324
3      324
Name: count, dtype: int64

In [29]:
track_df['program_number'].values[0]

'6  '

# ad-hoc analysis

In [14]:
project_utils.analyze_dataframe(start_df.head(2000), verbose=False)

,feature,missing_count,missing_ratio,value_types,mean,median,min,25%,75%,max,std
0,track_id,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,race_date,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,race_number,0,0,[int],5,5,1,3,7,10,2
3,program_number,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,weight_carried,0,0,[int],120,121,110,119,122,127,3
5,jockey,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,odds,0,0,[int],1526,660,5,280,1720,16800,2137
7,position_at_finish,0,0,[int],4,4,1,2,6,12,2


In [17]:
project_utils.analyze_dataframe(race_df.head(2000), verbose=False)

,feature,missing_count,missing_ratio,value_types,mean,median,min,25%,75%,max,std
0,track_id,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,race_date,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,race_number,0,0,[int],5,5,1,3,7,13,3
3,distance_id,0,0,[int],726,700,450,600,800,2000,148
4,course_type,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,track_condition,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,run_up_distance,0,0,[int],66,54,0,45,76,280,36
7,race_type,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,purse,0,0,[int],82520,62000,16000,42000,80000,1500000,106112
9,post_time,0,0,[int],407,340,100,225,509,1259,280


In [18]:
project_utils.analyze_dataframe(track_df.head(2000), verbose=False)

,feature,missing_count,missing_ratio,value_types,mean,median,min,25%,75%,max,std
0,track_id,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,race_date,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,race_number,0,0,[int],6,6,3,3,9,10,3
3,program_number,0,0,[str],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,trakus_index,0,0,[int],143,142,1,70,213,369,86
5,latitude,0,0,[float],41,41,41,41,41,43,1
6,longitude,0,0,[float],-74,-74,-74,-74,-74,-74,0


In [23]:
print(track_df.columns.tolist())

['track_id', 'race_date', 'race_number', 'program_number', 'trakus_index', 'latitude', 'longitude']
